# Two-phase momentum advection verification: translating inviscid droplet without surface tension 

![title](translating-droplet-figures/translating-droplet-3D-standalone-1.png)

A droplet is translated in a rectangular domain using a spatially constant internal and inlet velocity field. Length dimensions are in meters.

### Goal: Ensure the advection of the interface is consistent with the advection of the momentum

Advecting the two-phase momentum with a prescribed velocity $\mathbf{v}$ is modeled as

$\partial_t (\rho \mathbf{v}) + \nabla \cdot (\rho\mathbf{v}\mathbf{v}) = 0$

with $\mathbf{v} = \text{const} = \mathbf{v}_c = \mathbf{v}_f$ on the boundary and inside the solution domain. If discretized explicitly using Euler temporal discretization $\mathbf{v}=\text{const}$ gives 

$\mathbf{v}_c^{n+1} = \frac{\mathbf{v}_c^n(\rho_c^n - \frac{\Delta t}{|\Omega_c|}\sum_{f \in F_c}\rho_f^n \mathbf{v}_f^n\cdot \mathbf{S}_f)}{\rho_c^{n+1}}$

Since the passive transport of the droplet without viscous, hydrostatic, and surface tension forces (any forces for that matter) working on it, should not cause the accelleration of the fluid, $\mathbf{v}_c^{n+1} = \mathbf{v}_c^{n}$, so 

$\rho_c^{n+1} =\rho_c^n - \frac{\Delta t}{|\Omega_c|}\sum_{f \in F_c}\rho_f^n \mathbf{v}_f^n\cdot \mathbf{S}_f$ 

**which means the cell-centered densities must be updated using the same face-centered momentum flux $\rho_f^n \mathbf{v}_f^n\cdot \mathbf{S}_f$ - this is the momentum flux consistency requirement for the conservative formulation of the momentum equation in the single-field Navier-Stokes equations.**

**If the numerical method does not satisfy this condition, passive transport of a droplet causes artificial accelerations.**

**The accellerations appear as a source-term on the r.h.s. of the pressure equation - that drives $\nabla_c \cdot \mathbf{v}_c^{n+1} = 0$**

In [ ]:
import sys
#print(sys.path)
#sys.path.remove('/work/projects/project01456/lent/cases/scripts/modules')
print(sys.path)
import dataframeWithMetadata as dfmd
import dataAgglomeration as da
import os
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import rcParams
rcParams["figure.dpi"] = 400
sys.path.append(os.environ['ARGO_PYTHON_MODULES'])
#if (not(os.path.exists('figures'))):
#    os.mkdir('figures')

idx = pd.IndexSlice
    
def plot_density_ratio_dframe(dframe, tScheme, divScheme, title):
    """Plots the temporal evolution of the L-INFINITY velocity error norm for each mesh resolution and density ratio."""
    
    dframe=dframe.loc[idx[tScheme, divScheme]]
    resolutions = dframe.index.get_level_values("resolution").unique()
    CFLs = dframe.index.get_level_values("CFL_num").unique()
    #CFLs=[0.005, 0.01, 0.05, 0.1]
   # resolutions = [32]
    v = 0.01

    for resolution in resolutions:
        for CFL in CFLs:
            df_subset = dframe.loc[CFL,resolution,:]
            plt.plot(df_subset["time"], df_subset["Linf velocity error"] / v, 
                     label="RESOLUTION:%s; CFL:%s" % (resolution, CFL))
    plt.title(title, y= 1.08)
    plt.ylabel(r"$L_{\infty}(\frac{\|\mathbf{v}_c^e - \mathbf{v}_c^n\|}{\|\mathbf{v}_c^e\|})$")
               #"L_2 velocity error") 
               #"$L_{\infty}(\frac{\|\mathbf{v}_c^e - \mathbf{v}_c^n\|}{\|\mathbf{v}_c^e\|})$")
    plt.xlabel("Time in seconds")
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, shadow=True, ncol=2)

In [ ]:
agglomerator = da.data_agglomerator("mercuryAirDropletTranslation.parameter","stationaryDropletResults.csv", 
                                     "./interIsoFoam-mercuryAirDropletTranslation_00000_templateCase")
agglomerator.show_failed_variations()
agglomerator.write_agglomerated_study_data("interIsoFoam.mercuryAirDropletTranslation.csv")
dframe = agglomerator.study_dataframe()
dframe = dframe.sort_index()
#dframe = dframe.loc[(dframe.index.get_level_values('CFL_num') <= 0.01) & (dframe.index.get_level_values('resolution') <= 64)]

In [ ]:
plot_density_ratio_dframe(dframe,'Euler','upwind','interIsoFoam')

In [ ]:
plot_density_ratio_dframe(dframe,'Euler','limitedLinearV1','interIsoFoam')

In [ ]:
plot_density_ratio_dframe(dframe,'CrankNicolson0.9','upwind','interIsoFoam')

In [ ]:
plot_density_ratio_dframe(dframe,'CrankNicolson0.9','limitedLinearV1','interIsoFoam')

In [ ]:
agglomerator = da.data_agglomerator("mercuryAirDropletTranslation.parameter","stationaryDropletResults.csv", 
                                     "./interIsoRhoFoam-mercuryAirDropletTranslation_00000_templateCase")
agglomerator.show_failed_variations()
agglomerator.write_agglomerated_study_data("interIsoRhoFoam.mercuryAirDropletTranslation.csv")
dframe = agglomerator.study_dataframe()
dframe = dframe.sort_index()
#dframe = dframe.loc[(dframe.index.get_level_values('CFL_num') <= 0.01) & (dframe.index.get_level_values('resolution') <= 64)]


In [ ]:
plot_density_ratio_dframe(dframe, 'Euler', 'upwind', 'interIsoRhoFoam')

In [ ]:
plot_density_ratio_dframe(dframe, 'Euler','limitedLinearV1','interIsoRhoFoam')

In [ ]:
plot_density_ratio_dframe(dframe, 'CrankNicolson0.9', 'upwind', 'interIsoRhoFoam')

In [ ]:
plot_density_ratio_dframe(dframe, 'CrankNicolson0.9','limitedLinearV1','interIsoRhoFoam')